<a href="https://colab.research.google.com/github/masaki21/homework-day3/blob/main/Copy_of_ai_engineering_03_T4_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [1]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 81, done.
remote: Total 81 (delta 0), reused 0 (delta 0), pack-reused 81 (from 2)
Receiving objects: 100% (81/81), 1.34 MiB | 15.95 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [3]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import random
random.seed(0)

In [6]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig


model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [7]:
def generate_output(query):
  messages = [
      {"role": "user", "content": query},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

In [8]:
question =  "筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"
response = generate_output(question)
print(response)

 
 筋肉を大きくするには、**筋トレと食事の両方が重要**です。 

**筋トレ**は筋肉の成長を促しますが、**食事**は筋肉の成長に必要な栄養素を供給します。

**筋トレの効果**

* **筋肉の成長:**  筋トレは筋肉の繊維を成長させることで、筋肉の量と強さを増します。
* **代謝の向上:**  筋肉は代謝を促進する役割を果たします。
* **骨格の安定:**  筋肉は骨を支え、安定性を高めます。

**食事の効果**

* **タンパク質:**  筋肉の成長に必要なタンパク質を供給します。
* **炭水化物:**  エネルギーを供給し、筋肉の成長に必要です。
* **脂質:**  ホルモンの生成や細胞の機能に必要です。
* **ビタミン・ミネラル:**  筋肉の成長と修復に必要です。

**食事と筋トレの組み合わせ**

* **タンパク質:**  筋トレ後、筋肉の修復と成長に必要なタンパク質を積極的に摂取しましょう。
* **炭水化物:**  筋トレの後に、エネルギーを供給する炭


- 数値的な評価も見てみます。RagasにはAnswer Accuracyという評価指標があります。今回はこちらを参考に実装した評価関数を利用して測っていきます。

- 今回はgemmaでは性能が不安定だったので、OpenAIのgpt-4oで評価していきます。従って、scoreの実行はopenAI APIキーを所持している関心がある方のみで良いです。

In [ ]:
!pip install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.3/661.3 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.76.0
    Uninstalling openai-1.76.0:
      Successfully uninstalled openai-1.76.0


In [ ]:
# @title 評価実装
# gold_answer = "「Inference Time Scaling」とは、推論時に計算量を増やしてモデルの性能を高める手法です。これはモデルのサイズを大きくする代わりに、難しい入力に対して多くの計算リソースを使うことで、より良い出力を得ようとするアプローチです。"

# from openai import OpenAI
# from google.colab import userdata
# client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# def openai_generator(query):

#         messages = [
#                     {
#                         "role": "user",
#                         "content": query
#                     }
#                 ]

#         response = client.chat.completions.create(
#             model="gpt-4o-mini",
#             messages=messages
#         )
#         return response.choices[0].message.content

# def evaluate_answer_accuracy(query, response, reference):

#     template_accuracy1 = (
#           "Instruction: You are a world class state of the art assistant for rating "
#           "a User Answer given a Question. The Question is completely answered by the Reference Answer.\n"
#           "Say 4, if User Answer is full contained and equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 2, if User Answer is partially contained and almost equivalent to Reference Answer"
#           "in all terms, topics, numbers, metrics, dates and units.\n"
#           "Say 0, if User Answer is not contained in Reference Answer or not accurate in all terms, topics,"
#           "numbers, metrics, dates and units or the User Answer do not answer the question.\n"
#           "Do not explain or justify your rating. Your rating must be only 4, 2 or 0 according to the instructions above.\n"
#           "Even small discrepancies in meaning, terminology, directionality, or implication must result in a lower score. Only rate 4 if the User Answer is a complete and precise match to the Reference Answer in every aspect.\n"
#           "### Question: {query}\n"
#           "### {answer0}: {sentence_inference}\n"
#           "### {answer1}: {sentence_true}\n"
#           "The rating is:\n"
#       )
#     template_accuracy2 = (
#           "I will rate the User Answer in comparison to the Reference Answer for a given Question.\n"
#           "A rating of 4 indicates that the User Answer is entirely consistent with the Reference Answer, covering all aspects, topics, numbers, metrics, dates, and units.\n"
#           "A rating of 2 signifies that the User Answer is mostly aligned with the Reference Answer, with minor discrepancies in some areas.\n"
#           "A rating of 0 means that the User Answer is either inaccurate, incomplete, or unrelated to the Reference Answer, or it fails to address the Question.\n"
#           "I will provide the rating without any explanation or justification, adhering to the following scale: 0 (no match), 2 (partial match), 4 (exact match).\n"
#           "Even minor inconsistencies in meaning, terminology, emphasis, or factual detail should prevent a rating of 4. Only assign a 4 if the User Answer exactly and unambiguously matches the Reference Answer in every respect."
#           "Do not explain or justify my rating. My rating must be only 4, 2 or 0 only.\n\n"
#           "Question: {query}\n\n"
#           "{answer0}: {sentence_inference}\n\n"
#           "{answer1}: {sentence_true}\n\n"
#           "Rating: "
#       )

#     score1 = openai_generator(
#                 template_accuracy1.format(
#                       query=query,
#                       answer0="User Answer",
#                       answer1="Reference Answer",
#                       sentence_inference=response,
#                       sentence_true=reference,
#                     )
#                 )
#     try:
#       score1 = int(score1)
#     except:
#       print("Failed")
#       score1 = 0

#     score2 = openai_generator(
#                 template_accuracy2.format(
#                         query=query,
#                         answer0="Reference Answer",
#                         answer1="User Answer",
#                         sentence_inference=reference,
#                         sentence_true=response,
#                     )
#                   )

#     try:
#       score2 = int(score2)
#     except:
#       print("Failed")
#       score2 = 0


#     return (score1 + score2) / 2

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0


## 結果 (ベースモデル)

「google/gemma-2-2b-jpn-it」は「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用

## 2.1 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [9]:
!pip install -U langchain-community
!pip install sentence_transformers langchain
from langchain.embeddings import HuggingFaceEmbeddings

# 【現在】INF-Retrieverを使っている場合
# embeddings = HuggingFaceEmbeddings(model_name="infly/inf-retriever-v1-1.5b",
#                                    model_kwargs={"device": "cuda", "trust_remote_code": True})

# 【代替1】軽量なMiniLMモデルに差し替え
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2",
#                                    model_kwargs={"device": "cuda"})

# 【代替2】高精度なE5-largeモデルに差し替え
emb_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2",
                                   model_kwargs={"device": "cuda"})


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00


<ipython-input-9-29c3d2135cdf>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_model = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2",


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [10]:
with open("/content/lecture-ai-engineering/day3/data/筋肉栄養学.txt", "r") as f:
  raw_writedown = f.read()

In [ ]:
raw_writedown

'今回は筋肉栄養学シリーズの特別編として、大谷翔平選手の食習慣に関するニュースをもとに、体づくりやパフォーマンス向上に役立つ栄養の考え方を解説します。\nある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです。\nただし、その特別メニューの詳細は公開されていません。\n日ハム時代からも、大谷選手は登板日に「関東風うどん」をオーダーしていたようです。\nこれは「関東（地域）」と「完投（試合を最後まで投げ切る）」をかけた験担ぎでもありますが、単なるゲン担ぎではないと考えられます。\nおそらく彼はパフォーマンスを意識した食事を選んでいるのだと思います。\n「うどん＝小麦＝筋肉に悪い」という誤解をよく耳にします。\nしかし小麦が体づくりの敵というのは誤りです。\nもちろん小麦アレルギーなど個別の問題がある人は避けるべきですが、小麦自体がすべての人のボディメイクを妨げるというのは正しくありません。\n実際、人類の発展を支えてきた主食のひとつが小麦です。\nうどんは主に炭水化物で構成されており、脂質はほとんど含まれていません。\nこれは短時間・高強度の運動に必要なエネルギー源として非常に優れており、筋トレや試合前の食事に適しています。\n特に白いうどんは消化吸収が早く、エネルギーとしてすぐに使われる特徴があります。\nエナジーゼリーのような感覚で活用できると言えるでしょう。\n大谷選手のようにピッチングもバッティングもこなす選手は、エネルギー消費が非常に大きいため、試合前に糖質を補給する「カーボローディング」が理にかなっています。\nこれは炭水化物を事前に多く摂ることで、筋肉内にグリコーゲンとして糖質を貯め、試合中の持久力や集中力を支える方法です。\nマラソンやサッカー選手などもこの方法で高強度の運動に備えます。\nグリコーゲンローディングには主に2つの方法があります。\n1つは炭水化物を一度減らしてグリコーゲンを枯渇させた後、大量に摂取する方法です。\nもう1つは食事制限をせず、糖質の摂取量を増やしていく方法です。\n現在ではスポーツパフォーマンスの現場で後者の方法が多く採用されています。\nボディメイクの世界では前者の方法も根強く使われています。\n小麦が悪者扱いされるの

In [11]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[20])

ドキュメントサイズ:  223
ドキュメントの例: 
 現在ではスポーツパフォーマンスの現場で後者の方法が多く採用されています


In [12]:
import torch, numpy as np
question = "筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"

# ---- 質問・ドキュメントの前処理（E5 は prefix 推奨） ----
e5_query     = "query: "   + question
e5_documents = ["passage: " + d for d in documents]

# ---- 埋め込み計算 ----
query_vec     = torch.tensor(emb_model.embed_query(e5_query))            # shape = (768,)
doc_vecs      = torch.tensor(emb_model.embed_documents(e5_documents))    # shape = (N, 768)

# ---- コサイン類似度 ----
scores = (query_vec @ doc_vecs.T) / (query_vec.norm() * doc_vecs.norm(dim=1))
scores = scores.cpu().numpy() * 100       # 好みでスケール

# ---- 上位 k 件を確認 ----
topk = 5
top_idx = scores.argsort()[::-1][:topk]
for rank, idx in enumerate(top_idx, 1):
    print(f"[{rank}] score={scores[idx]:.1f} → {documents[idx][:80]}…")

[1] score=85.9 → 食べ物を食べるとは何なのか…
[2] score=85.6 → 食べることは、体を作り、動かし、調整し、維持するために必要不可欠なのです…
[3] score=85.6 → 食べることで当然、栄養素が体に取り込まれます…
[4] score=85.4 → 筋トレをすると筋肉に血液が集まり、胃腸への血流が減るため、消化吸収の機能も落ちます…
[5] score=85.4 → 豆はあまり注目されないかもしれませんが、よく噛んで食べることで満腹感が得られ、咀嚼によってエネルギー消費も高まります…


In [ ]:
# Retrievalの実行
question = "筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

ValueError: Prompt name 'query' not found in the configured prompts dictionary with keys [].

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 89.31804656982422)
ある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです 


取得したドキュメント2: (Score: 89.15975189208984)
大谷選手が登板日に食べる「関東風うどん」は、験担ぎだけでなく、消化の良い炭水化物を中心とした高パフォーマンス向けの食事である可能性が高いと考えています 


取得したドキュメント3: (Score: 88.72749328613281)
日ハム時代からも、大谷選手は登板日に「関東風うどん」をオーダーしていたようです 


取得したドキュメント4: (Score: 83.77871704101562)
今回は筋肉栄養学シリーズの特別編として、大谷翔平選手の食習慣に関するニュースをもとに、体づくりやパフォーマンス向上に役立つ栄養の考え方を解説します 


取得したドキュメント5: (Score: 82.9944839477539)
大谷選手のようにピッチングもバッティングもこなす選手は、エネルギー消費が非常に大きいため、試合前に糖質を補給する「カーボローディング」が理にかなっています 




In [13]:
references = "\n".join(["* " + documents[i] for i in scores.argsort()[::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"
response = generate_output(query)
print(response)

 筋肉を大きくするには、**「食べること」も非常に重要**です。 

参考資料では、筋トレによる筋肉の成長と、消化吸収の機能の低下について説明されています。 

筋トレは筋肉の成長を促しますが、**筋肉の成長に必要な栄養素は、食事から摂取する必要があります**。 

* **筋肉を作るためには、タンパク質、炭水化物、脂質などの栄養素が必須**です。
* **筋肉の成長に必要な栄養素を十分に摂取しないと、筋肉の成長が遅くなり、効果が期待できません**。


 





In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

0.0


### 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

## 2.2 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)
---

<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。

---
<修正後>
---

講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**:
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**:
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [ ]:
with open("/content/lecture-ai-engineering/day3/data/筋肉栄養学.txt", "r") as f:
  raw_writedown = f.read()

In [14]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[50])

ドキュメントサイズ:  223
ドキュメントの例: 
 それぞれが持つエネルギーの合計が、食品の総カロリーとなります


In [ ]:
# Retrievalの実行
question = "筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'encode'

In [ ]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 89.31804656982422)
ある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです 


取得したドキュメント2: (Score: 89.15975189208984)
大谷選手が登板日に食べる「関東風うどん」は、験担ぎだけでなく、消化の良い炭水化物を中心とした高パフォーマンス向けの食事である可能性が高いと考えています 


取得したドキュメント3: (Score: 88.72749328613281)
日ハム時代からも、大谷選手は登板日に「関東風うどん」をオーダーしていたようです 


取得したドキュメント4: (Score: 83.77871704101562)
今回は筋肉栄養学シリーズの特別編として、大谷翔平選手の食習慣に関するニュースをもとに、体づくりやパフォーマンス向上に役立つ栄養の考え方を解説します 


取得したドキュメント5: (Score: 82.9944839477539)
大谷選手のようにピッチングもバッティングもこなす選手は、エネルギー消費が非常に大きいため、試合前に糖質を補給する「カーボローディング」が理にかなっています 




In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 大谷翔平選手が試合前に「関東風うどん」を食べるのはなぜですか？"
response = generate_output(query)
print(response)

記事によると、大谷選手が試合前に「関東風うどん」を食べているのは、**験担ぎだけでなく、消化の良い炭水化物を中心とした高パフォーマンス向けの食事である可能性が高い**と考えられています。


 



In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時に計算資源を増やすこと」という概念を正確に捉えるようになった

### 問題点
* 「Pretraining時」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---



# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [15]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"
response = generate_output(query)
print(response)

IndexError: invalid index to scalar variable.

In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇




# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [16]:
!pip install -U langchain-community langchain sentence_transformers faiss-cpu
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder

from langchain.schema import Document
docs = [Document(page_content=txt) for txt in documents]

# prefix を付ける helper（E5 では query と passage で prefix が必要）
def add_passage_prefix(docs):
    for d in docs:
        d.page_content = "passage: " + d.page_content
    return docs

docs_prefixed = add_passage_prefix(docs)

# FAISS に登録
vector_store = FAISS.from_documents(docs_prefixed, emb_model)

# ④ 質問を入力 → ベクトル検索 → CrossEncoder で再ランク
# -------------------------------------------------
question = "筋トレだけで筋肉はつきますか？筋肉を大きくするには「食べること」も必要なのでしょうか？"
e5_query = "query: " + question   # E5 用 prefix

# 上位 10 件を取得
candidate_docs = vector_store.similarity_search(e5_query, k=10)

# CrossEncoder で関連度再スコア
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2", device="cuda")

pairs  = [(question, doc.page_content) for doc in candidate_docs]
scores = reranker.predict(pairs)      # ndarray shape=(10,)

# スコアで降順ソート → 上位 3 件を使用
top_k = 3
scored_docs   = sorted(zip(candidate_docs, scores), key=lambda x: x[1], reverse=True)
reranked_docs = [doc for doc, _ in scored_docs[:top_k]]

# -------------------------------------------------
# ⑤ LLM に問い合わせ
# -------------------------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Gemma-2B の準備（量子化は元コードと同じ）
model_name = "google/gemma-2-2b-jpn-it"
tokenizer  = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
llm        = AutoModelForCausalLM.from_pretrained(
                model_name, device_map="auto",
                quantization_config=bnb_config, torch_dtype=torch.bfloat16)

def generate_output(prompt):
    messages = [{"role": "user", "content": prompt}]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True,
                                              return_tensors="pt").to(llm.device)
    outputs = llm.generate(input_ids, max_new_tokens=256, do_sample=False,
                           eos_token_id=[tokenizer.eos_token_id,
                                         tokenizer.convert_tokens_to_ids("<|eot_id|>")])
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

# 参考資料を結合してプロンプト作成
context_text = "\n".join(doc.page_content.replace("passage: ", "") for doc in reranked_docs)
prompt = f"[参考資料]\n{context_text}\n\n[質問] {question}"

answer = generate_output(prompt)
print(answer)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.6 MB/s eta 0:00:00


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  その質問は非常に興味深いですね！ 

筋トレだけで筋肉は増えることは事実ですが、**筋肉を大きくするには「食べること」も非常に重要**です。

**筋トレと食事の組み合わせは、筋肉の成長を最大限に引き出すための重要な要素です。**

* **筋トレ:**  筋肉を作るための基礎となる要素です。
* **食事:**  筋肉を作るための材料となる栄養素を供給します。

**栄養素の役割**

* **タンパク質:**  筋肉の構成要素であり、筋トレ後の修復と成長に不可欠です。
* **炭水化物:**  エネルギー源となり、筋トレ後の回復をサポートします。
* **脂質:**  ホルモンの生成やエネルギー産生に役立ちます。

**豆類の例**

豆類は、タンパク質や鉄分を多く含み、満腹感を得やすく、エネルギー消費を促進する効果があります。

**まとめ**

筋トレと食事の両面で努力を続けることで、より効果的な筋肉の成長を目指せます。






In [17]:
!pip install -q transformers accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct")
model     = AutoModelForCausalLM.from_pretrained("NousResearch/Meta-Llama-3-8B-Instruct",
                                                 device_map="auto", torch_dtype="auto")

system_message = (
    "あなたは回答の品質を評価する専門家です。"
    "与えられた質問とそれに対する2つの回答を比較し、"
    "正確性、完全性、関連性の観点で優れている方を判断し、理由を述べてください。"
)
user_message = (
    f"質問: {question}\n\n"
    f"回答A: {response}\n\n"
    f"回答B: {answer}\n\n"
    "各回答を正確性、完全性、関連性で5点満点評価し、"
    "より優れた回答がどちらかを選び、その理由を詳しく説明してください。"
)

prompt = (
    f"{system_message}\n\n"
    f"{user_message}\n\n"
    "### 回答:"
)

inputs  = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [ ]:
!pip install -U openai
from openai import OpenAI

client = OpenAI(api_key="")   # ★ここにキー

system_message = (
    "あなたは回答の品質を評価する専門家です。"
    "与えられた質問とそれに対する2つの回答を比較し、"
    "正確性、完全性、関連性の観点で優れている方を判断し、理由を述べてください。"
)
user_message = (
    f"質問: {question}\n\n"
    f"回答A: {response}\n\n"
    f"回答B: {answer}\n\n"
    "各回答を正確性、完全性、関連性で5点満点評価し、"
    "より優れた回答がどちらかを選び、その理由を詳しく説明してください。"
)

resp = client.chat.completions.create(
    model="gpt-4o-mini",        # 例: GPT-4o mini・gpt-4o など
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user",   "content": user_message}
    ]
)

evaluation = resp.choices[0].message.content
print(evaluation)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import openai
openai.api_key = ""

system_message = (
    "あなたは回答の品質を評価する専門家です。"
    "与えられた質問とそれに対する2つの回答を比較し、"
    "正確性、完全性、関連性の観点で優れている方を判断し、理由を述べてください。"
)
user_message = (
    f"質問: {question}\n\n"
    f"回答A: {response}\n\n"
    f"回答B: {answer}\n\n"
    "各回答を正確性、完全性、関連性で5点満点評価し、より優れた回答がどちらかを選び、その理由を詳しく説明してください。"
)

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[{"role": "system", "content": system_message},
              {"role": "user", "content": user_message}]
)
evaluation = response['choices'][0]['message']['content']
print(evaluation)


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:

  query = f"与えられた[参考資料]が[質問]に直接関連しているかを、'yes''no'で答えること。[参考資料]\n{ref}\n\n[質問] 大谷翔平選手が試合前に「関東風うどん」を食べるのはなぜですか？"
  response = generate_output(query)

  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)



対象となるドキュメント:
 今回は筋肉栄養学シリーズの特別編として、大谷翔平選手の食習慣に関するニュースをもとに、体づくりやパフォーマンス向上に役立つ栄養の考え方を解説します。
ある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです。
ただし、その特別メニューの詳細は公開されていません

関連しているかどうか:  yes 





対象となるドキュメント:
 私自身、現在イスラエルでピタパンをよく食べています。
脂質を控えながら炭水化物を摂取することで、トレーニングのパフォーマンスを支えています。
大谷選手が登板日に食べる「関東風うどん」は、験担ぎだけでなく、消化の良い炭水化物を中心とした高パフォーマンス向けの食事である可能性が高いと考えています。
筋トレやスポーツのパフォーマンスを発揮するためには、炭水化物の戦略的な摂取が必要です

関連しているかどうか:  yes 





対象となるドキュメント:
 ある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです。
ただし、その特別メニューの詳細は公開されていません。
日ハム時代からも、大谷選手は登板日に「関東風うどん」をオーダーしていたようです。
これは「関東（地域）」と「完投（試合を最後まで投げ切る）」をかけた験担ぎでもありますが、単なるゲン担ぎではないと考えられます

関連しているかどうか:  yes 





対象となるドキュメント:
 今回は筋肉栄養学シリーズの特別編として、大谷翔平選手の食習慣に関するニュースをもとに、体づくりやパフォーマンス向上に役立つ栄養の考え方を解説します。
ある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです

関連しているかどうか:  yes 





対象となるドキュメント:
 特に白いうどんは消化吸収が早く、エネルギーとしてすぐに使われる特徴があります。
エナジーゼリーのような感覚で活用できると言えるでしょう。
大谷選手のようにピッチングもバッティングもこな

In [ ]:
print(len(references))

4


上記より、上位4件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。（生成内容が確立的なので、4件でない可能性もあります）

In [ ]:
query =  f"[参考資料]\n{references}\n\n[質問] 大谷翔平選手が試合前に「関東風うどん」を食べるのはなぜですか？"
response = generate_output(query)
print(response)

大谷選手が試合前に「関東風うどん」を食べる理由は、いくつかの可能性が考えられます。

* **験担ぎ:**  「関東風うどん」は、地域と完投をかけた験担ぎとして、試合の成功を祈願するものです。
* **消化の良い炭水化物:**  試合前に食べる「関東風うどん」は、消化の良い炭水化物を中心とした高パフォーマンス向けの食事である可能性があります。
* **特別なメニュー:**  その特別メニューの詳細については公開されていませんが、試合前の食事に特別な意味を持たせている可能性があります。



 





In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)

2.0


## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

**注意事項**

**ここから先のセルを実行した場合、GPUメモリ不足になる可能性が高いです。**


このノートブックでは、GPUはT4を使用しています。
Colab Pro等を契約し、L4などのよりGPUメモリの大きいものを使用するか、モデルやその設定等を変更するなどの工夫が必要になります。

In [ ]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[5])

ドキュメントサイズ:  55
ドキュメントの例: 
 このプロテインの中身である「ホエイペプチド」は、アミノ酸が連なってできたタンパク質を酵素で分解し、ジペプチド、トリペプチドといった形になったものです。これらは消化を必要とせず、そのまま吸収できます。したがって、吸収が非常に早く、EAA（必須アミノ酸）と同等のスピードで体内に取り込まれます。


In [ ]:
emb_model = SentenceTransformer("intfloat/e5-small-v2")


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
question = "大谷翔平選手が試合前に「関東風うどん」を食べるのはなぜですか？"

# query_embeddings = emb_model.encode([question], prompt_name="query")
# document_embeddings = emb_model.encode(documents)

query_embeddings = emb_model.encode(question, normalize_embeddings=True)
document_embeddings = emb_model.encode(documents, normalize_embeddings=True)
scores = (query_embeddings @ document_embeddings.T)


scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[89.82425689697266, 86.98045349121094, 84.97310638427734, 86.11186981201172, 84.94342041015625, 82.91117858886719, 83.29666137695312, 84.58100128173828, 82.54106903076172, 85.71757507324219, 86.06718444824219, 85.0556640625, 82.86044311523438, 85.08832550048828, 84.51387023925781, 85.53959655761719, 85.49287414550781, 88.79679870605469, 82.29090118408203, 83.365478515625, 83.76295471191406, 87.61123657226562, 85.761474609375, 86.75119018554688, 86.73892211914062, 83.27081298828125, 85.60105895996094, 85.67346954345703, 87.32621765136719, 87.26055908203125, 87.44508361816406, 86.73536682128906, 90.12662506103516, 86.38748931884766, 86.3220443725586, 85.33748626708984, 85.51445770263672, 85.3469009399414, 83.25072479248047, 85.318603515625, 83.77642822265625, 87.87515258789062, 86.7508316040039, 86.05065155029297, 87.35553741455078, 88.67720031738281, 85.89607238769531, 86.41419219970703, 87.23986053466797, 87.44194793701172, 86.86351776123047, 83.88391876220703, 86.4053955078125, 85.017

In [ ]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

IndexError: invalid index to scalar variable.

In [ ]:
import numpy as np
topk = 2

# ---- 1. スコアを numpy 1D 配列にしておく（既に numpy なら不要）----
scores = np.asarray(scores)        # torch.Tensor の場合は scores.cpu().numpy()

# ---- 2. 大きい順に上位 topk 個のインデックスを取得 ----
topk_idx = np.argsort(-scores)[:topk]   # −を付けて降順に

# ---- 3. 取り出して表示 ----
for rank, idx in enumerate(topk_idx, 1):
    print(f"取得したドキュメント{rank}: (Score: {scores[idx]:.2f})")
    print(documents[idx], "\n")


取得したドキュメント1: (Score: 90.13)
日本でよく食べられる豆といえば大豆ですが、私が現在いるイスラエルでは大豆が簡単には手に入りません。その代わり、さまざまな豆に出会うことができました。今回のシリーズでは、そうした豆の情報を皆さんと共有したいと思います。 

取得したドキュメント2: (Score: 89.82)
今回は筋肉栄養学シリーズの特別編として、大谷翔平選手の食習慣に関するニュースをもとに、体づくりやパフォーマンス向上に役立つ栄養の考え方を解説します。 ある記事によると、大谷選手は通訳・水原一平さんの父親がシェフを務めるアメリカの高級和食店に通っており、試合前には特別な「関東風うどん」を食べているとのことです。 ただし、その特別メニューの詳細は公開されていません。 日ハム時代からも、大谷選手は登板日に「関東風うどん」をオーダーしていたようです。 これは「関東（地域）」と「完投（試合を最後まで投げ切る）」をかけた験担ぎでもありますが、単なるゲン担ぎではないと考えられます。 おそらく彼はパフォーマンスを意識した食事を選んでいるのだと思います。 「うどん＝小麦＝筋肉に悪い」という誤解をよく耳にします。 しかし小麦が体づくりの敵というのは誤りです。 もちろん小麦アレルギーなど個別の問題がある人は避けるべきですが、小麦自体がすべての人のボディメイクを妨げるというのは正しくありません。 実際、人類の発展を支えてきた主食のひとつが小麦です。 うどんは主に炭水化物で構成されており、脂質はほとんど含まれていません。 これは短時間・高強度の運動に必要なエネルギー源として非常に優れており、筋トレや試合前の食事に適しています。 特に白いうどんは消化吸収が早く、エネルギーとしてすぐに使われる特徴があります。 エナジーゼリーのような感覚で活用できると言えるでしょう。 大谷選手のようにピッチングもバッティングもこなす選手は、エネルギー消費が非常に大きいため、試合前に糖質を補給する「カーボローディング」が理にかなっています。 これは炭水化物を事前に多く摂ることで、筋肉内にグリコーゲンとして糖質を貯め、試合中の持久力や集中力を支える方法です。 マラソンやサッカー選手などもこの方法で高強度の運動に備えます。 グリコーゲンローディングには主に2つの方法があります。 1つは炭水化

In [ ]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
query =  f"[参考資料]\n{references}\n\n[質問] 大谷翔平選手が試合前に「関東風うどん」を食べるのはなぜですか？"
response = generate_output(query)
print(response)

IndexError: invalid index to scalar variable.

In [ ]:
import numpy as np

topk = 2                   # 既に定義済みなら不要
scores_arr = np.asarray(scores)           # torch→numpy 変換（既に numpy ならそのまま）

# ---- 上位 k 件のインデックスを取得（降順）----
topk_idx = scores_arr.argsort()[::-1][:topk]

# ---- 参考資料を文字列に整形 ----
reference_text = "\n".join(f"* {documents[i]}" for i in topk_idx)

# ---- 質問プロンプトを合成 ----
query = (
    "[参考資料]\n"
    f"{reference_text}\n\n"
    "[質問] 大谷翔平選手が試合前に「関東風うどん」を食べるのはなぜですか？"
)

response = generate_output(query)
print(response)

記事によると、大谷選手が試合前に「関東風うどん」を食べているのは、**パフォーマンスを意識した食事**として考えられています。 

特に、うどんは**消化の良い炭水化物**で構成されており、試合中の持久力や集中力を支えるためのエネルギー源として最適であると考えられています。





In [ ]:
# # 評価
# score = evaluate_answer_accuracy(question, response, gold_answer)
# print(score)